In [2]:
from geo_loc_info import get_nearby_establishments

In [22]:
import sqlite3

# Connect to the SQLite database
db_path = "alerts.db"  # Ensure this path is correct
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get all column names from the 'alert' table
cursor.execute("PRAGMA table_info(alerts);")
columns = [col[1] for col in cursor.fetchall()]
print(columns)

# Identify latitude and longitude columns
lat_col = next((col for col in columns if "latitude" in col.lower()), None)
lon_col = next((col for col in columns if "longitude" in col.lower()), None)

if lat_col and lon_col:
    # Fetch all latitude and longitude values
    cursor.execute(f"SELECT {lat_col}, {lon_col} FROM alerts;")
    lat_lon_data = cursor.fetchall()

    # Print results
    i=1
    for lat, lon in lat_lon_data:
        print(f"Latitude: {lat}, Longitude: {lon}")
        near_establishments=get_nearby_establishments(lat=lat,lon=lon)
        print(near_establishments[0]["name"])
        geo_loc=""
        for establishment in near_establishments:
            geo_loc = geo_loc + f"{establishment["name"]}" + f" ({establishment["type"]}) ,"
        # near_establishments
        # comma_separated_str = ", ".join(map(str, near_establishments))

        update_alert(i,"geo_info",geo_loc)
        i+=1

else:
    print("No latitude/longitude columns found in the 'alert' table.")

# Close the connection
conn.close()


['id', 'type', 'severity', 'location', 'latitude', 'longitude', 'timestamp', 'description', 'image', 'video_path', 'geo_info', 'responder_type', 'response_time', 'resolution_time', 'status']
Latitude: 24.467, Longitude: 54.365
Unnamed Place
Updated 'geo_info' of alert ID 1 to 'Unnamed Place (hospital) ,دجاج كنتاكي (fast_food) ,دجاج كنتاكي (fast_food) ,Texas Chicken (restaurant) ,First Abu Dhabi Bank (FAB) (bank) ,'.
Latitude: 24.4669, Longitude: 54.3424
La Brioche
Updated 'geo_info' of alert ID 2 to 'La Brioche (cafe) ,تيم هورتونز (cafe) ,Unnamed Place (parking) ,Unnamed Place (toilets) ,Cold Stone Creamery (ice_cream) ,'.
Latitude: 24.4699, Longitude: 54.3296
Byky Sport & Leisure
Updated 'geo_info' of alert ID 3 to 'Byky Sport & Leisure (bicycle_rental) ,La Terrazza (restaurant) ,Vasco's (restaurant) ,Special Restaurant & Cafe (restaurant) ,Bocca (restaurant) ,'.
Latitude: 24.4993, Longitude: 54.3858
Brauhaus
Updated 'geo_info' of alert ID 4 to 'Brauhaus (pub) ,Trader Vic's (restauran

In [14]:
def update_alert(alert_id, column, new_value):
    """Updates a specific column in the alerts table for a given id."""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Ensure column name is valid
    valid_columns = ["type", "severity", "location", "latitude", "longitude", "timestamp", "description", "image", "video_path", "geo_info", "responder_type", "response_time", "resolution_time", "status"]
    if column not in valid_columns:
        print(f"Error: '{column}' is not a valid column name.")
        return
    
    # Create query and execute update
    query = f"UPDATE alerts SET {column} = ? WHERE id = ?"
    cursor.execute(query, (new_value, alert_id))
    
    conn.commit()
    conn.close()
    print(f"Updated '{column}' of alert ID {alert_id} to '{new_value}'.")